### feature 대상으로 LGBM 학습  
(Developed from dacon_etri_base_mod1.ipynb)

In [38]:
from datetime import datetime

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530


submission_folder = '/users/KTL/Desktop/dacon/submission/'
submission_file = f'submission_final_mod4_5_{timestamp}.csv'


In [39]:

import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import torch
import tensorflow as tf


In [40]:
# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

# 파일 경로 설정 - VSCode 상대경로로 변경
# 실제 경로에 맞게 수정 필요
base_folder =  '/users/KTL/Desktop/ETRI_lifelog_dataset'
folder = '/ch2025_data_items'

data_dir = base_folder + folder 


# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 

In [41]:
# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

✅ Loaded: mACStatus, shape = (939896, 3)
✅ Loaded: mActivity, shape = (961062, 3)
✅ Loaded: mAmbience, shape = (476577, 3)
✅ Loaded: mBle, shape = (21830, 3)
✅ Loaded: mGps, shape = (800611, 3)
✅ Loaded: mLight, shape = (96258, 3)
✅ Loaded: mScreenStatus, shape = (939653, 3)
✅ Loaded: mUsageStats, shape = (45197, 3)
✅ Loaded: mWifi, shape = (76336, 3)
✅ Loaded: wHr, shape = (382918, 3)
✅ Loaded: wLight, shape = (633741, 3)
✅ Loaded: wPedo, shape = (748100, 9)


In [42]:
# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

In [43]:
# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 


In [44]:
# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

⏳ mACStatus 분리 중...
✅ mACStatus_test → (335849, 3), mACStatus_train → (604047, 3)
⏳ mActivity 분리 중...
✅ mActivity_test → (343579, 3), mActivity_train → (617483, 3)
⏳ mAmbience 분리 중...
✅ mAmbience_test → (170453, 3), mAmbience_train → (306124, 3)
⏳ mBle 분리 중...
✅ mBle_test → (8140, 3), mBle_train → (13690, 3)
⏳ mGps 분리 중...
✅ mGps_test → (287386, 3), mGps_train → (513225, 3)
⏳ mLight 분리 중...
✅ mLight_test → (34439, 3), mLight_train → (61819, 3)
⏳ mScreenStatus 분리 중...
✅ mScreenStatus_test → (336160, 3), mScreenStatus_train → (603493, 3)
⏳ mUsageStats 분리 중...
✅ mUsageStats_test → (16499, 3), mUsageStats_train → (28698, 3)
⏳ mWifi 분리 중...
✅ mWifi_test → (27467, 3), mWifi_train → (48869, 3)
⏳ wHr 분리 중...
✅ wHr_test → (143311, 3), wHr_train → (239607, 3)
⏳ wLight 분리 중...
✅ wLight_test → (233809, 3), wLight_train → (399932, 3)
⏳ wPedo 분리 중...
✅ wPedo_test → (288832, 9), wPedo_train → (459268, 9)


In [45]:
def process_mACStatus(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df = df.sort_values(['subject_id', 'timestamp']) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_charging'].values # 0/1 상태 
        times = group['timestamp'].values # 충전 상태 비율 
        ratio_charging = status.mean() 
        # 상태 전이 횟수 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        lengths = [] 
        current_len = 0 
        for val in status: 
            if val == 1: 
                current_len += 1 
            elif current_len > 0: 
                lengths.append(current_len) 
                current_len = 0 
        if current_len > 0: 
            lengths.append(current_len) 
        avg_charging_duration = np.mean(lengths) if lengths else 0 
        max_charging_duration = np.max(lengths) if lengths else 0 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'charging_ratio': ratio_charging, 
            'charging_transitions': transitions, 
            'avg_charging_duration': avg_charging_duration, 
            'max_charging_duration': max_charging_duration, 
        }) 
    return pd.DataFrame(results) 

mACStatus_df2 = process_mACStatus(mACStatus_df) 

In [46]:
def process_mActivity(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        counts = group['m_activity'].value_counts(normalize=True) # 비율 
        row = {'subject_id': subj, 'date': date} 
        # 0~8 비율 저장 
        for i in range(9): 
            row[f'activity_{i}_ratio'] = counts.get(i, 0) 
        # 주요 활동 정보 
        row['dominant_activity'] = group['m_activity'].mode()[0] 
        row['num_unique_activities'] = group['m_activity'].nunique() 
        summary.append(row) 
    return pd.DataFrame(summary) 

mActivity_df2 = process_mActivity(mActivity_df) 

In [47]:
# 지정된 10개 라벨 
top_10_labels = [ 
    "Inside, small room", "Speech", "Silence", "Music", "Narration, monologue", 
    "Child speech, kid speaking", "Conversation", "Speech synthesizer", "Shout", "Babbling" 
] 

def process_mAmbience_top10(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    # 초기화 
    for label in top_10_labels + ['others']: 
        df[label] = 0.0 
    for idx, row in df.iterrows(): 
        parsed = ast.literal_eval(row['m_ambience']) if isinstance(row['m_ambience'], str) else row['m_ambience'] 
        others_prob = 0.0 
        for label, prob in parsed: 
            prob = float(prob) 
            if label in top_10_labels: 
                df.at[idx, label] = prob 
            else: 
                others_prob += prob 
        df.at[idx, 'others'] = others_prob 
    return df.drop(columns=['m_ambience']) 

mAmbience_df2= process_mAmbience_top10(mAmbience_df) 

In [48]:
def summarize_mAmbience_daily(df): 
    prob_cols = [col for col in df.columns if col not in ['subject_id', 'timestamp', 'date']] 
    # 하루 단위로 평균값 요약 
    daily_summary = df.groupby(['subject_id', 'date'])[prob_cols].mean().reset_index() 
    return daily_summary 

mAmbience_df2 = summarize_mAmbience_daily(mAmbience_df2) 


In [49]:
def process_mBle(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble'] 
        rssi_list = [] 
        class_0_cnt = 0 
        class_other_cnt = 0 
        for device in entry: 
            try: 
                rssi = int(device['rssi']) 
                rssi_list.append(rssi) 
                if str(device['device_class']) == '0': 
                    class_0_cnt += 1 
                else: 
                    class_other_cnt += 1 
            except: 
                continue # malformed record 
        feature = { 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'device_class_0_cnt': class_0_cnt, 
            'device_class_others_cnt': class_other_cnt, 
            'device_count': len(rssi_list), 
            'rssi_mean': np.mean(rssi_list) if rssi_list else np.nan, 
            'rssi_min': np.min(rssi_list) if rssi_list else np.nan, 
            'rssi_max': np.max(rssi_list) if rssi_list else np.nan, 
        } 
        features.append(feature) 
    return pd.DataFrame(features) 

In [50]:
def summarize_mBle_daily(df): 
    # row 단위 BLE feature 추출 
    df = process_mBle(df) 
    # 하루 단위로 cnt 합치기 
    grouped = df.groupby(['subject_id', 'date']).agg({ 
        'device_class_0_cnt': 'sum', 
        'device_class_others_cnt': 'sum', 
        'rssi_mean': 'mean', 
        'rssi_min': 'min', 
        'rssi_max': 'max', 
    }).reset_index() 
    # 총합 구해서 비율 계산 
    total_cnt = grouped['device_class_0_cnt'] + grouped['device_class_others_cnt'] 
    grouped['device_class_0_ratio'] = grouped['device_class_0_cnt'] / total_cnt.replace(0, np.nan) 
    grouped['device_class_others_ratio'] = grouped['device_class_others_cnt'] / total_cnt.replace(0, np.nan) 
    # 필요 없는 원래 cnt 컬럼 제거 
    grouped.drop(columns=['device_class_0_cnt', 'device_class_others_cnt'], inplace=True) 
    return grouped 

mBle_df2 = summarize_mBle_daily(mBle_df) 

In [51]:
def process_mGps(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        gps_list = ast.literal_eval(row['m_gps']) if isinstance(row['m_gps'], str) else row['m_gps'] 
        altitudes = [] 
        latitudes = [] 
        longitudes = [] 
        speeds = [] 
        for entry in gps_list: 
            try: 
                altitudes.append(float(entry['altitude'])) 
                latitudes.append(float(entry['latitude'])) 
                longitudes.append(float(entry['longitude'])) 
                speeds.append(float(entry['speed'])) 
            except: 
                continue 
        features.append({ 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'altitude_mean': np.mean(altitudes) if altitudes else np.nan, 
            'latitude_std': np.std(latitudes) if latitudes else np.nan, 
            'longitude_std': np.std(longitudes) if longitudes else np.nan, 
            'speed_mean': np.mean(speeds) if speeds else np.nan, 
            'speed_max': np.max(speeds) if speeds else np.nan, 
            'speed_std': np.std(speeds) if speeds else np.nan, 
        }) 
    return pd.DataFrame(features) 

In [52]:
m_Gps_df2 = process_mGps(mGps_df) 
m_Gps_df2 = m_Gps_df2.groupby(['subject_id', 'date']).agg({ 
    'altitude_mean': 'mean', 
    'latitude_std': 'mean', 
    'longitude_std': 'mean', 
    'speed_mean': 'mean', 
    'speed_max': 'max', 
    'speed_std': 'mean' 
}).reset_index() 

In [53]:
def process_mLight(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['hour'] = df['timestamp'].dt.hour 
    # 밤(22~05시), 낮(06~21시) 구분 
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6) 
    # 하루 단위 요약 
    daily = df.groupby(['subject_id', 'date']).agg( 
        light_mean=('m_light', 'mean'), 
        light_std=('m_light', 'std'), 
        light_max=('m_light', 'max'), 
        light_min=('m_light', 'min'), 
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()), 
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()), 
        light_night_ratio=('is_night', 'mean') # 밤 시간 측정 비율 
    ).reset_index() 
    return daily 

mLight_df2 = process_mLight(mLight_df) 

In [54]:
def process_mScreenStatus(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_screen_use'].values 
        ratio_on = status.mean() 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        durations = [] 
        current = 0 
        for val in status: 
            if val == 1: 
                current += 1 
            elif current > 0: 
                durations.append(current) 
                current = 0 
        if current > 0: 
            durations.append(current) 
        features.append({ 
            'subject_id': subj, 
            'date': date, 
            'screen_on_ratio': ratio_on, 
            'screen_on_transitions': transitions, 
            'screen_on_duration_avg': np.mean(durations) if durations else 0, 
            'screen_on_duration_max': np.max(durations) if durations else 0, 
        }) 
    return pd.DataFrame(features) 

mScreenStatus_df2 = process_mScreenStatus(mScreenStatus_df) 


In [55]:
top_apps = [ 
    'One UI 홈', '카카오톡', '시스템 UI', 'NAVER', '캐시워크', 
    '성경일독Q', 'YouTube', '통화', '메시지', '타임스프레드', 'Instagram'
] 

def process_mUsageStats(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        app_time = {app: 0 for app in top_apps} 
        others_time = 0 
        for row in group['m_usage_stats']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for entry in parsed: 
                app = entry.get('app_name') 
                time = entry.get('total_time', 0) 
                if app in top_apps: 
                    app_time[app] += int(time) 
                else: 
                    others_time += int(time) 
        feature = { 
            'subject_id': subj, 
            'date': date, 
            'others_time': others_time 
        } 
        # 각 앱별 컬럼 추가 
        feature.update({f'{app}_time': app_time[app] for app in top_apps}) 
        features.append(feature) 
    return pd.DataFrame(features) 

mUsageStats_df2 = process_mUsageStats(mUsageStats_df) 

In [56]:

def process_mWifi(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        rssi_all = [] 
        for row in group['m_wifi']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for ap in parsed: 
                try: 
                    rssi = int(ap['rssi']) 
                    rssi_all.append(rssi) 
                except: 
                    continue 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'wifi_rssi_mean': np.mean(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_min': np.min(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_max': np.max(rssi_all) if rssi_all else np.nan, 
            'wifi_detected_cnt': len(rssi_all) 
        }) 
    return pd.DataFrame(results) 

mWifi_df2 = process_mWifi(mWifi_df) 

In [57]:

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wHr_df2 = process_wHr_by_timeblock(wHr_df) 

In [58]:

def process_wLight_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            lux = block_group['w_light'].dropna().values 
            if len(lux) == 0: 
                continue 
            block_stats[f'wlight_{block}_mean'] = np.mean(lux) 
            block_stats[f'wlight_{block}_std'] = np.std(lux) 
            block_stats[f'wlight_{block}_max'] = np.max(lux) 
            block_stats[f'wlight_{block}_min'] = np.min(lux) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wLight_df2 = process_wLight_by_timeblock(wLight_df)

In [59]:
def process_wPedo(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = df.groupby(['subject_id', 'date']).agg({ 
        'step': 'sum', 
        'step_frequency': 'mean', 
        'distance': 'sum', 
        'speed': ['mean', 'max'], 
        'burned_calories': 'sum' 
    }).reset_index() 
    # 컬럼 이름 정리 
    summary.columns = ['subject_id', 'date', 'step_sum', 'step_frequency_mean', 'distance_sum', 'speed_mean', 'speed_max', 'burned_calories_sum'] 
    return summary 

wPedo_df2 = process_wPedo(wPedo_df) 

In [60]:

from functools import reduce 
df_list = [ 
    mACStatus_df2, 
    mActivity_df2, 
    mAmbience_df2, 
    mBle_df2, 
    m_Gps_df2, 
    mLight_df2, 
    mScreenStatus_df2, 
    mUsageStats_df2, 
    mWifi_df2, 
    wHr_df2, 
    wLight_df2, 
    wPedo_df2 
] 

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list) 


In [61]:

# metrics_train의 lifelog_date → datetime.date 형으로 변환 
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date 

# merged_df의 date도 변환 
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date 

# 1. date 기준 정렬을 위해 metrics_train의 lifelog_date -> date로 맞추기 
metrics_train_renamed = metrics_train.rename(columns={'lifelog_date': 'date'}) 

# 2. train_df: metrics_train과 일치하는 (subject_id, date) → 라벨 포함 
train_df = pd.merge(metrics_train_renamed, merged_df, on=['subject_id', 'date'], how='inner') 

# 3. test_df: metrics_train에 없는 (subject_id, date) 
merged_keys = merged_df[['subject_id', 'date']] 
train_keys = metrics_train_renamed[['subject_id', 'date']] 
test_keys = pd.merge(merged_keys, train_keys, on=['subject_id', 'date'], how='left', indicator=True) 
test_keys = test_keys[test_keys['_merge'] == 'left_only'].drop(columns=['_merge']) 
test_df = pd.merge(test_keys, merged_df, on=['subject_id', 'date'], how='left') 


In [62]:

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

# ✅ feature 준비 
X = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']) 
X.fillna(0, inplace=True) # 결측값 처리 

test_X = test_df.drop(columns=['subject_id', 'date']) 
test_X.fillna(0, inplace=True) 


In [63]:

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True) # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True) # 연속된 _ 제거 
        .str.strip("_") # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X = sanitize_column_names(X) 
test_X = sanitize_column_names(test_X) 

In [64]:
# 결과 저장 
# 그룹별 중요 특성 선택
important_features = [
    # 스크린 관련 특성
    'screen_on_ratio', 'screen_on_duration_avg', 'screen_on_duration_max',
    
    # 와이파이 신호 관련 특성
    'wifi_rssi_max', 'wifi_rssi_mean', 'wifi_detected_cnt',
    
    # 활동 관련 특성
    'activity_3_ratio', 'activity_4_ratio',
    
    # 충전 관련 특성
    'charging_ratio', 'max_charging_duration', 'avg_charging_duration',
    
    # 신호 관련 특성
    'rssi_mean',
    
    # 빛 관련 특성
    'light_night_mean', 'light_max', 'light_std',
    
    # 위치/움직임 관련 특성
    'altitude_mean', 'speed_max_x',
    
    # 앱 사용 관련 특성
    '메시지_time', 'others_time', 'Narration_monologue'
]

# 선택된 특성만으로 데이터셋 구성
X_selected = X[important_features]
test_X_selected = test_X[important_features]



In [65]:
# LightGBM 라이브러리 임포트
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping 

# GridSearch를 위한 교차 검증 설정
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}



In [66]:
# 먼저 이 셀을 실행해서 필요한 라이브러리를 설치하고 import 하세요
import sys
import subprocess

# Optuna 설치 (아직 설치되지 않은 경우)
try:
    import optuna
    print("✅ Optuna가 이미 설치되어 있습니다.")
except ImportError:
    print("🔄 Optuna 설치 중...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "optuna"])
    import optuna
    print("✅ Optuna 설치 완료!")

# 필요한 모든 라이브러리 import
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("✅ 모든 라이브러리 import 완료!")

# ------------------ 베이지안 최적화 함수 정의 ------------------
def optimize_lgbm_bayesian(X, y, task_type='binary', n_trials=50, study_name=None):
    """
    LightGBM 베이지안 최적화
    """
    
    def objective(trial):
        # 하이퍼파라미터 범위 정의
        params = {
            'objective': 'binary' if task_type == 'binary' else 'multiclass',
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
            'num_leaves': trial.suggest_int('num_leaves', 10, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': -1
        }
        
        # 다중 분류인 경우 클래스 수 추가
        if task_type == 'multiclass':
            params['num_class'] = len(np.unique(y))
        
        # 모델 생성 및 교차검증
        model = LGBMClassifier(**params)
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
        # 점수 계산
        scoring = 'f1' if task_type == 'binary' else 'f1_macro'
        scores = cross_val_score(model, X, y, cv=cv, scoring=scoring, n_jobs=-1)
        
        return scores.mean()
    
    # Optuna 스터디 생성 (TPESampler 사용)
    sampler = TPESampler(seed=42)
    study = optuna.create_study(
        direction='maximize',
        sampler=sampler,
        study_name=study_name or f'lgbm_{task_type}_optimization'
    )
    
    # 최적화 실행
    print(f"🔍 {study_name or 'LightGBM'} 베이지안 최적화 시작 ({n_trials}회 시도)")
    
    # 진행률 표시를 위한 콜백 함수
    def callback(study, trial):
        if trial.number % 10 == 0:
            print(f"   시도 {trial.number}: 현재 최고 점수 = {study.best_value:.4f}")
    
    study.optimize(objective, n_trials=n_trials, callbacks=[callback])
    
    # 결과 출력
    print(f"✅ 최적화 완료!")
    print(f"   최고 점수: {study.best_value:.4f}")
    print(f"   최적 파라미터: {study.best_params}")
    
    return study.best_params, study.best_value

print("✅ 베이지안 최적화 함수 정의 완료!")


✅ Optuna가 이미 설치되어 있습니다.
✅ 모든 라이브러리 import 완료!
✅ 베이지안 최적화 함수 정의 완료!


In [67]:
print("\n===== 오토인코더를 사용한 고급 특성 추출 =====")

# 하이퍼파라미터 설정
encoding_dim = min(64, X_selected.shape[1])  # 인코딩 차원 (원본 특성 수보다 작게 설정)
batch_size = 64
epochs = 100

# 입력 차원
input_dim = X_selected.shape[1]

# 오토인코더 모델 구축
input_layer = Input(shape=(input_dim,))

# 인코더 부분
encoded = Dense(128, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.3)(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# 디코더 부분
decoded = Dense(64, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.3)(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# 전체 오토인코더 모델
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 인코더 부분만 추출 (특성 추출용)
encoder = Model(input_layer, encoded)



===== 오토인코더를 사용한 고급 특성 추출 =====


In [68]:
# 훈련/검증 세트 분리
X_train_auto, X_val_auto = train_test_split(X_selected, test_size=0.2, random_state=42)

# 조기 종료 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 오토인코더 모델 학습
print("오토인코더 학습 중...")
autoencoder.fit(
    X_train_auto, X_train_auto,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(X_val_auto, X_val_auto),
    callbacks=[early_stopping],
    verbose=1
)

# 인코딩된 특성 추출
print("인코딩된 특성 추출 중...")
encoded_features_train = encoder.predict(X_selected)
encoded_features_test = encoder.predict(test_X_selected)

# 원본 특성과 인코딩된 특성 결합
X_combined = np.hstack([X_selected, encoded_features_train])
test_X_combined = np.hstack([test_X_selected, encoded_features_test])

print(f"원본 특성 수: {X_selected.shape[1]}")
print(f"인코딩된 특성 수: {encoded_features_train.shape[1]}")
print(f"결합된 특성 수: {X_combined.shape[1]}")


오토인코더 학습 중...
Epoch 1/100
6/6 [==============================] - 1s 37ms/step - loss: 7942339821568.0000 - val_loss: 9015309369344.0000
Epoch 2/100
6/6 [==============================] - 0s 15ms/step - loss: 7942339297280.0000 - val_loss: 9015308320768.0000
Epoch 3/100
6/6 [==============================] - 0s 14ms/step - loss: 7942339297280.0000 - val_loss: 9015308320768.0000
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 7942339821568.0000 - val_loss: 9015308320768.0000
Epoch 5/100
6/6 [==============================] - 0s 10ms/step - loss: 7942339297280.0000 - val_loss: 9015308320768.0000
Epoch 6/100
6/6 [==============================] - 0s 10ms/step - loss: 7942339821568.0000 - val_loss: 9015308320768.0000
Epoch 7/100
6/6 [==============================] - 0s 10ms/step - loss: 7942339297280.0000 - val_loss: 9015308320768.0000
Epoch 8/100
6/6 [==============================] - 0s 7ms/step - loss: 7942339297280.0000 - val_loss: 9015308320768.0000
Epoch 9/100

In [69]:
# ------------------ 베이지안 최적화로 이진 분류 모델 학습 ------------------
print("\\n===== 베이지안 최적화를 이용한 이진 분류 모델 학습 =====")

binary_preds_combined = {}
binary_best_params = {}
binary_best_scores = {}

for col in targets_binary:
    print(f"\\n{'='*50}")
    print(f"📊 {col} 이진 분류 베이지안 최적화")
    print(f"{'='*50}")
    
    y = train_df[col]
    
    # 베이지안 최적화 실행
    best_params, best_score = optimize_lgbm_bayesian(
        X_combined, y, 
        task_type='binary', 
        n_trials=30,  # 처음엔 30회로 시작 (나중에 늘려도 됨)
        study_name=f"{col}_binary"
    )
    
    # 최적 파라미터로 모델 생성 및 예측
    best_model = LGBMClassifier(**best_params)
    best_model.fit(X_combined, y)
    binary_preds_combined[col] = best_model.predict(test_X_combined)
    
    # 결과 저장
    binary_best_params[col] = best_params
    binary_best_scores[col] = best_score
    
    print(f"✅ {col} 최적화 완료 - 최고 F1 점수: {best_score:.4f}")

print("\\n🎉 모든 이진 분류 모델 베이지안 최적화 완료!")

# 이진 분류 결과 요약
print("\\n===== 이진 분류 베이지안 최적화 결과 요약 =====")
for col in targets_binary:
    print(f"{col}: F1-Score = {binary_best_scores[col]:.4f}")

[I 2025-05-30 13:59:11,049] A new study created in memory with name: Q1_binary


\n===== 베이지안 최적화를 이용한 이진 분류 모델 학습 =====
\n==================================================
📊 Q1 이진 분류 베이지안 최적화
🔍 Q1_binary 베이지안 최적화 시작 (30회 시도)


[I 2025-05-30 13:59:14,481] Trial 0 finished with value: 0.5350153555398063 and parameters: {'learning_rate': 0.03574712922600244, 'n_estimators': 1907, 'num_leaves': 223, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.5350153555398063.


   시도 0: 현재 최고 점수 = 0.5350


[I 2025-05-30 13:59:16,081] Trial 1 finished with value: 0.5252257726594644 and parameters: {'learning_rate': 0.11114989443094977, 'n_estimators': 139, 'num_leaves': 292, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 0 with value: 0.5350153555398063.
[I 2025-05-30 13:59:18,230] Trial 2 finished with value: 0.519934077026874 and parameters: {'learning_rate': 0.04345454109729477, 'n_estimators': 653, 'num_leaves': 188, 'max_depth': 4, 'min_child_samples': 33, 'subsample': 0.6831809216468459, 'colsample_bytree': 0.728034992108518, 'reg_alpha': 0.7851759613930136, 'reg_lambda': 0.19967378215835974}. Best is trial 0 with value: 0.5350153555398063.
[I 2025-05-30 13:59:20,598] Trial 3 finished with value: 0.5324019463211629 and parameters: {'learning_rate': 0.05748924681991978, 'n_estimators': 1226, 'num_leaves': 23, 'max_depth': 10, 'min_child

...

   시도 10: 현재 최고 점수 = 0.7950


[I 2025-05-30 14:01:03,630] Trial 11 finished with value: 0.7972649939493653 and parameters: {'learning_rate': 0.010233276814276507, 'n_estimators': 1465, 'num_leaves': 92, 'max_depth': 15, 'min_child_samples': 100, 'subsample': 0.8513496163789993, 'colsample_bytree': 0.6422284320908205, 'reg_alpha': 0.011046564754964586, 'reg_lambda': 0.9989171937915848}. Best is trial 11 with value: 0.7972649939493653.
[I 2025-05-30 14:01:04,162] Trial 12 finished with value: 0.7973680621236615 and parameters: {'learning_rate': 0.010267583312981183, 'n_estimators': 1327, 'num_leaves': 93, 'max_depth': 15, 'min_child_samples': 100, 'subsample': 0.8588729908849949, 'colsample_bytree': 0.6114239841499921, 'reg_alpha': 0.03358588493447212, 'reg_lambda': 0.9920380779070681}. Best is trial 12 with value: 0.7973680621236615.
[I 2025-05-30 14:01:04,613] Trial 13 finished with value: 0.731508439004931 and parameters: {'learning_rate': 0.29596284206078866, 'n_estimators': 1124, 'num_leaves': 97, 'max_depth': 1

   시도 20: 현재 최고 점수 = 0.7974


[I 2025-05-30 14:01:11,197] Trial 21 finished with value: 0.7826546506453048 and parameters: {'learning_rate': 0.013054160544440772, 'n_estimators': 1702, 'num_leaves': 69, 'max_depth': 14, 'min_child_samples': 89, 'subsample': 0.8338120110550713, 'colsample_bytree': 0.6460839172068573, 'reg_alpha': 0.13336771843704237, 'reg_lambda': 0.9961367392345147}. Best is trial 12 with value: 0.7973680621236615.
[I 2025-05-30 14:01:11,914] Trial 22 finished with value: 0.7912043071403708 and parameters: {'learning_rate': 0.013782210527620286, 'n_estimators': 1264, 'num_leaves': 136, 'max_depth': 14, 'min_child_samples': 93, 'subsample': 0.79010195265003, 'colsample_bytree': 0.6583391668233065, 'reg_alpha': 0.21730356309239884, 'reg_lambda': 0.750159389169004}. Best is trial 12 with value: 0.7973680621236615.
[I 2025-05-30 14:01:12,981] Trial 23 finished with value: 0.7794910343468778 and parameters: {'learning_rate': 0.0248466512698118, 'n_estimators': 1590, 'num_leaves': 83, 'max_depth': 12, 'm

✅ 최적화 완료!
   최고 점수: 0.7997
   최적 파라미터: {'learning_rate': 0.010809067308626146, 'n_estimators': 1053, 'num_leaves': 50, 'max_depth': 15, 'min_child_samples': 90, 'subsample': 0.8211338116630437, 'colsample_bytree': 0.6103450665213718, 'reg_alpha': 0.39522516860502016, 'reg_lambda': 0.9082528588305049}
[LightGBM] [Info] Number of positive: 298, number of negative: 152
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3732
[LightGBM] [Info] Number of data points in the train set: 450, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.662222 -> initscore=0.673213
[LightGBM] [Info] Start training from score 0.673213
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [70]:
# ------------------ 베이지안 최적화로 다중 분류 모델 학습 ------------------
print(f"\\n{'='*50}")
print(f"📊 S1 다중 분류 베이지안 최적화")
print(f"{'='*50}")

y_multi = train_df['S1']
n_classes = len(y_multi.unique())

# 베이지안 최적화 실행
multi_best_params, multi_best_score = optimize_lgbm_bayesian(
    X_combined, y_multi,
    task_type='multiclass',
    n_trials=30,
    study_name="S1_multiclass"
)

# 최적 파라미터로 모델 생성 및 예측
best_multi_model = LGBMClassifier(**multi_best_params)
best_multi_model.fit(X_combined, y_multi)
multiclass_pred_combined = best_multi_model.predict(test_X_combined)

print(f"✅ S1 최적화 완료 - 최고 F1 macro 점수: {multi_best_score:.4f}")


[I 2025-05-30 14:01:17,068] A new study created in memory with name: S1_multiclass


\n==================================================
📊 S1 다중 분류 베이지안 최적화
🔍 S1_multiclass 베이지안 최적화 시작 (30회 시도)


[I 2025-05-30 14:01:20,397] Trial 0 finished with value: 0.3695845295278533 and parameters: {'learning_rate': 0.03574712922600244, 'n_estimators': 1907, 'num_leaves': 223, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.3695845295278533.


   시도 0: 현재 최고 점수 = 0.3696


[I 2025-05-30 14:01:21,114] Trial 1 finished with value: 0.3517873485308402 and parameters: {'learning_rate': 0.11114989443094977, 'n_estimators': 139, 'num_leaves': 292, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 0 with value: 0.3695845295278533.
[I 2025-05-30 14:01:23,347] Trial 2 finished with value: 0.37900732191858677 and parameters: {'learning_rate': 0.04345454109729477, 'n_estimators': 653, 'num_leaves': 188, 'max_depth': 4, 'min_child_samples': 33, 'subsample': 0.6831809216468459, 'colsample_bytree': 0.728034992108518, 'reg_alpha': 0.7851759613930136, 'reg_lambda': 0.19967378215835974}. Best is trial 2 with value: 0.37900732191858677.
[I 2025-05-30 14:01:25,013] Trial 3 finished with value: 0.3515199366480019 and parameters: {'learning_rate': 0.05748924681991978, 'n_estimators': 1226, 'num_leaves': 23, 'max_depth': 10, 'min_ch

   시도 10: 현재 최고 점수 = 0.4216


[I 2025-05-30 14:01:39,031] Trial 11 finished with value: 0.4349019769578471 and parameters: {'learning_rate': 0.2957379384814452, 'n_estimators': 1196, 'num_leaves': 93, 'max_depth': 3, 'min_child_samples': 100, 'subsample': 0.8513496163789993, 'colsample_bytree': 0.9835968465735538, 'reg_alpha': 0.616211739462904, 'reg_lambda': 0.00467357070277391}. Best is trial 11 with value: 0.4349019769578471.
[I 2025-05-30 14:01:40,847] Trial 12 finished with value: 0.4169041170727809 and parameters: {'learning_rate': 0.2681687322961067, 'n_estimators': 1327, 'num_leaves': 94, 'max_depth': 3, 'min_child_samples': 100, 'subsample': 0.8588729908849949, 'colsample_bytree': 0.9055786136308785, 'reg_alpha': 0.5444237593561123, 'reg_lambda': 0.003107199514773005}. Best is trial 11 with value: 0.4349019769578471.
[I 2025-05-30 14:01:42,747] Trial 13 finished with value: 0.42009851327804554 and parameters: {'learning_rate': 0.29596284206078866, 'n_estimators': 1503, 'num_leaves': 103, 'max_depth': 6, 'm

   시도 20: 현재 최고 점수 = 0.4349


[I 2025-05-30 14:01:57,113] Trial 21 finished with value: 0.4098097066409703 and parameters: {'learning_rate': 0.27137722764035593, 'n_estimators': 1295, 'num_leaves': 83, 'max_depth': 3, 'min_child_samples': 94, 'subsample': 0.8928816965088971, 'colsample_bytree': 0.9961525785372274, 'reg_alpha': 0.6810578081241968, 'reg_lambda': 0.08810026062906123}. Best is trial 11 with value: 0.4349019769578471.
[I 2025-05-30 14:01:58,646] Trial 22 finished with value: 0.4238421654981574 and parameters: {'learning_rate': 0.2085968356242911, 'n_estimators': 1201, 'num_leaves': 136, 'max_depth': 4, 'min_child_samples': 72, 'subsample': 0.8112358360495093, 'colsample_bytree': 0.9487922371341003, 'reg_alpha': 0.6212904036835192, 'reg_lambda': 0.07064311434205794}. Best is trial 11 with value: 0.4349019769578471.
[I 2025-05-30 14:01:59,963] Trial 23 finished with value: 0.4290327638712001 and parameters: {'learning_rate': 0.2046756664613435, 'n_estimators': 920, 'num_leaves': 128, 'max_depth': 4, 'min_

✅ 최적화 완료!
   최고 점수: 0.4349
   최적 파라미터: {'learning_rate': 0.2957379384814452, 'n_estimators': 1196, 'num_leaves': 93, 'max_depth': 3, 'min_child_samples': 100, 'subsample': 0.8513496163789993, 'colsample_bytree': 0.9835968465735538, 'reg_alpha': 0.616211739462904, 'reg_lambda': 0.00467357070277391}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3732
[LightGBM] [Info] Number of data points in the train set: 450, number of used features: 29
[LightGBM] [Info] Start training from score -1.146403
[LightGBM] [Info] Start training from score -0.697602
[LightGBM] [Info] Start training from score -1.690407
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

In [71]:
# ------------------ 성능 평가 ------------------
print("\\n===== 베이지안 최적화 결과 성능 평가 =====")

# 다중 분류 성능 평가
X_train_comb, X_val_comb, y_train_multi, y_val_multi = train_test_split(
    X_combined, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

# 최적 파라미터로 검증용 모델 학습
eval_model_comb = LGBMClassifier(**multi_best_params)
eval_model_comb.fit(X_train_comb, y_train_multi)
y_pred_comb = eval_model_comb.predict(X_val_comb)

print("\\n=== S1 다중 분류 성능 평가 ===")
print(classification_report(y_val_multi, y_pred_comb))
print("\\n혼동 행렬:")
print(confusion_matrix(y_val_multi, y_pred_comb))

# 원본 특성과 결합 특성 비교
print("\\n===== 원본 특성 vs 결합 특성 모델 성능 비교 =====")

# 원본 특성으로 베이지안 최적화 (비교용)
print("원본 특성으로 베이지안 최적화 중...")
orig_best_params, orig_best_score = optimize_lgbm_bayesian(
    X_selected, y_multi,
    task_type='multiclass',
    n_trials=20,  # 비교용이므로 조금 적게
    study_name="S1_original_features"
)

X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(
    X_selected, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

eval_model_orig = LGBMClassifier(**orig_best_params)
eval_model_orig.fit(X_train_orig, y_train_orig)
y_pred_orig = eval_model_orig.predict(X_val_orig)

# 성능 비교
acc_orig = accuracy_score(y_val_orig, y_pred_orig)
acc_comb = accuracy_score(y_val_multi, y_pred_comb)
f1_orig = f1_score(y_val_orig, y_pred_orig, average='macro')
f1_comb = f1_score(y_val_multi, y_pred_comb, average='macro')

print(f"정확도 - 원본 특성: {acc_orig:.4f}, 결합 특성: {acc_comb:.4f}, 향상: {(acc_comb-acc_orig)*100:.2f}%")
print(f"F1 점수 - 원본 특성: {f1_orig:.4f}, 결합 특성: {f1_comb:.4f}, 향상: {(f1_comb-f1_orig)*100:.2f}%")


\n===== 베이지안 최적화 결과 성능 평가 =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000223 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2995
[LightGBM] [Info] Number of data points in the train set: 360, number of used features: 29
[LightGBM] [Info] Start training from score -1.149906
[LightGBM] [Info] Start training from score -0.698718
[LightGBM] [Info] Start training from score -1.681411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-05-30 14:02:09,785] A new study created in memory with name: S1_original_features


              precision    recall  f1-score   support

           0       0.38      0.38      0.38        29
           1       0.56      0.51      0.53        45
           2       0.30      0.38      0.33        16

    accuracy                           0.44        90
   macro avg       0.41      0.42      0.42        90
weighted avg       0.46      0.44      0.45        90

\n혼동 행렬:
[[11 12  6]
 [14 23  8]
 [ 4  6  6]]
\n===== 원본 특성 vs 결합 특성 모델 성능 비교 =====
원본 특성으로 베이지안 최적화 중...
🔍 S1_original_features 베이지안 최적화 시작 (20회 시도)


[I 2025-05-30 14:02:12,046] Trial 0 finished with value: 0.37829935268006243 and parameters: {'learning_rate': 0.03574712922600244, 'n_estimators': 1907, 'num_leaves': 223, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.37829935268006243.


   시도 0: 현재 최고 점수 = 0.3783


[I 2025-05-30 14:02:12,564] Trial 1 finished with value: 0.3710566483031223 and parameters: {'learning_rate': 0.11114989443094977, 'n_estimators': 139, 'num_leaves': 292, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 0 with value: 0.37829935268006243.
[I 2025-05-30 14:02:14,280] Trial 2 finished with value: 0.3777200554489621 and parameters: {'learning_rate': 0.04345454109729477, 'n_estimators': 653, 'num_leaves': 188, 'max_depth': 4, 'min_child_samples': 33, 'subsample': 0.6831809216468459, 'colsample_bytree': 0.728034992108518, 'reg_alpha': 0.7851759613930136, 'reg_lambda': 0.19967378215835974}. Best is trial 0 with value: 0.37829935268006243.
[I 2025-05-30 14:02:15,547] Trial 3 finished with value: 0.36316306139301097 and parameters: {'learning_rate': 0.05748924681991978, 'n_estimators': 1226, 'num_leaves': 23, 'max_depth': 10, 'min_c

   시도 10: 현재 최고 점수 = 0.4051


[I 2025-05-30 14:02:26,847] Trial 11 finished with value: 0.41775945618851024 and parameters: {'learning_rate': 0.2957379384814452, 'n_estimators': 1196, 'num_leaves': 93, 'max_depth': 3, 'min_child_samples': 100, 'subsample': 0.8513496163789993, 'colsample_bytree': 0.9835968465735538, 'reg_alpha': 0.616211739462904, 'reg_lambda': 0.00467357070277391}. Best is trial 11 with value: 0.41775945618851024.
[I 2025-05-30 14:02:28,197] Trial 12 finished with value: 0.3938556518460728 and parameters: {'learning_rate': 0.018626884233576644, 'n_estimators': 986, 'num_leaves': 104, 'max_depth': 6, 'min_child_samples': 78, 'subsample': 0.8588729908849949, 'colsample_bytree': 0.8859620061900357, 'reg_alpha': 0.583617765837666, 'reg_lambda': 0.003107199514773005}. Best is trial 11 with value: 0.41775945618851024.
[I 2025-05-30 14:02:30,113] Trial 13 finished with value: 0.4051309912237519 and parameters: {'learning_rate': 0.29596284206078866, 'n_estimators': 1503, 'num_leaves': 103, 'max_depth': 3, 

✅ 최적화 완료!
   최고 점수: 0.4246
   최적 파라미터: {'learning_rate': 0.1727305457165782, 'n_estimators': 932, 'num_leaves': 131, 'max_depth': 15, 'min_child_samples': 76, 'subsample': 0.7655620151757302, 'colsample_bytree': 0.8230258295337463, 'reg_alpha': 0.45889632946592745, 'reg_lambda': 0.34244034918267435}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2142
[LightGBM] [Info] Number of data points in the train set: 360, number of used features: 20
[LightGBM] [Info] Start training from score -1.149906
[LightGBM] [Info] Start training from score -0.698718
[LightGBM] [Info] Start training from score -1.681411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [72]:
# ------------------ 최적화 결과 요약 ------------------
print("\\n===== 베이지안 최적화 최종 결과 요약 =====")

print("\\n📊 이진 분류 결과:")
total_binary_improvement = 0
for col in targets_binary:
    print(f"  {col}: F1 = {binary_best_scores[col]:.4f}")
    total_binary_improvement += binary_best_scores[col]

avg_binary_score = total_binary_improvement / len(targets_binary)
print(f"  평균 F1 점수: {avg_binary_score:.4f}")

print(f"\\n📊 다중 분류 결과:")
print(f"  S1: F1 macro = {multi_best_score:.4f}")

print(f"\\n🔄 원본 vs 결합 특성 비교:")
print(f"  원본 특성 F1 macro: {orig_best_score:.4f}")
print(f"  결합 특성 F1 macro: {multi_best_score:.4f}")
improvement = (multi_best_score - orig_best_score) * 100
print(f"  오토인코더 개선 효과: {improvement:+.2f}%")

if improvement > 0:
    print("  ✅ 오토인코더 특성 결합이 성능을 향상시켰습니다!")
else:
    print("  ⚠️ 오토인코더 효과가 미미합니다. 다른 접근 방법을 고려해보세요.")

print("\\n🎯 베이지안 최적화 완료! GridSearch 대비 더 효율적인 최적화가 수행되었습니다.")

\n===== 베이지안 최적화 최종 결과 요약 =====
\n📊 이진 분류 결과:
  Q1: F1 = 0.5607
  Q2: F1 = 0.6594
  Q3: F1 = 0.7177
  S2: F1 = 0.7619
  S3: F1 = 0.7997
  평균 F1 점수: 0.6999
\n📊 다중 분류 결과:
  S1: F1 macro = 0.4349
\n🔄 원본 vs 결합 특성 비교:
  원본 특성 F1 macro: 0.4246
  결합 특성 F1 macro: 0.4349
  오토인코더 개선 효과: +1.03%
  ✅ 오토인코더 특성 결합이 성능을 향상시켰습니다!
\n🎯 베이지안 최적화 완료! GridSearch 대비 더 효율적인 최적화가 수행되었습니다.


In [73]:
# sample 기반 제출 포맷 가져오기
submission_final = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

# ID 만들기 (submission에서 예측한 결과와 연결하기 위해)
submission_final['ID'] = submission_final['subject_id'] + '_' + submission_final['lifelog_date'].astype(str)

In [74]:

# 예측 결과 연결할 수 있도록 동일한 순서로 정렬
# 보통 예측 결과는 test_df 기준이므로 정렬 보장되어야 함
assert len(submission_final) == len(multiclass_pred_combined) # shape 체크

# 다중 분류 예측 붙이기
submission_final['S1'] = multiclass_pred_combined

# 이진 분류 결과 붙이기
for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
    submission_final[col] = binary_preds_combined[col].astype(int) # 확률 아닌 class 예측


In [75]:

# 최종 제출 형식 정렬
submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

# 저장
submission_final.to_csv(submission_folder + submission_file, index=False)

# VSCode에서는 files.download()가 작동하지 않으므로 대체

print(f"✅ 제출 파일이 저장되었습니다: {os.path.abspath(submission_file)}")

✅ 제출 파일이 저장되었습니다: c:\Users\KTL\Desktop\submission_final_mod4_5_0530_1356.csv
